# `IMDB` 데이터 셋 예제
### 영화 리뷰를 사용한 텍스트 분류
- 총 5000건의 영화 리뷰 텍스트를 담은 `IMDB` 데이터 셋<br><br>
  - 훈련용 25000개, 테스트용 25000개로 구성되어 있음<br><br>
  - 긍정 리뷰는 1, 부정 리뷰는 0으로 나타냄<br><br>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [ ]:
print('훈련 샘플 : {} / 레이블 : {}'.format(len(train_data), len(train_labels)))

In [ ]:
print(train_data.shape, train_labels.shape, test_data.shape)

In [ ]:
train_data[0:2]
# 단어 하나씩 인덱스 번호를 지정해서 숫자로 글을 표현
# 겹치는 단어는 같은 인덱스 번호로, 나머지는 각각의 인덱스 번호로 표현

In [ ]:
train_labels[0:10]
# 긍정적인 리뷰 : 1 / 부정적인 리뷰 : 0

In [ ]:
print(len(train_data[0])) # 리뷰 길이 218
print(len(train_data[1])) # 리뷰 길이 189
print(len(train_data[2])) # 리뷰 길이 141
# 각각의 길이가 다르기 때문에 제일 긴 리뷰를 기준을고 동일하게 채워주는 작업을 진행해야 함

In [ ]:
word_index = imdb.get_word_index()

print(word_index.items())
# 각 단어와 인덱스 번호를 짝지어서 출력

In [ ]:
word_index = {k:(v + 3) for k, v in word_index.items()}

word_index
# k : 단어 대입 / v : 단어 인덱스 (3을 더해서 맨 앞의 3개의 번호 자리를 나중에 이용함)

In [ ]:
word_index['<PAD>'] = 0
word_index['<START>'] = 1
word_index['<UNK>'] = 2
word_index['<UNUSED>'] = 3
# 기본적으로 라이브러리를 이용한 단어 인덱스 번호 지정은 0이 아닌 1부터 시작함
# 0은 인덱스 번호가 처음에 없기 때문에 임의로 지정해서 새로운 단어를 넣어줌

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

reverse_word_index
# 인덱스 번호와 단어 순서 바꾸는 작업

In [ ]:
def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])

decode_review(train_data[0])
# 인덱스 번호에 해당하는 단어를 텍스트로 전환해주는 함수
# 해당 인덱스 번호의 단어가 없는 경우 ?로 출력

- `pad_sequences(sequences, maxlen=None, dtype='int32', padding='pre', truncating='pre', value=0.0)`<br><br>
  - 문장의 길이를 `maxlen` 인자로 맞추어 준다.<br><br>
  - 120으로 지정했다면 120보다 짧은 문장은 0으로 채워서 120 단어로 맞춰주고, 120보다 긴 문장은 120 단어까지만 잘라낸다.<br><br>
  - `(num_samples, num_timesteps)`으로 2차원의 `numpy` 배열로 만들어준다.<br><br>
  - `maxlen`을 120으로 지정했다면, `num_timesteps`도 120이 된다.<br><br>
  - 인수<br><br>
    - `padding` : `pre` or `post`<br><br>
    - `truncating` : `pre` or `post`<br><br>
    - `value` : 채워질 값 (`default=0`)<br><br>

In [ ]:
train_data = keras.preprocessing.sequence.pad_sequences(
  train_data,
  value=word_index['<PAD>'],
  padding='post', # 뒷부분 채우기
  maxlen=256
)

test_data = keras.preprocessing.sequence.pad_sequences(
  test_data,
  value=word_index['<PAD>'],
  padding='post',
  maxlen=256
)

train_data[0]

In [ ]:
vocab_size = 10000

model = keras.Sequential()

model.add(keras.layers.Embedding(
  vocab_size,
  16,
  input_shape=(None,)
))

model.add(keras.layers.GlobalAveragePooling1D())
# 평균값을 이용해 데이터 처리
# sequence 차원에 대해 평균을 계산하여 각 샘플에 대해 고정된 길이(여기에선 16)의 출력 벡터 반환

model.add(keras.layers.Dense(
  16,
  activation='relu'
))

model.add(keras.layers.Dense(
  1,
  activation='sigmoid'
))

model.summary()

In [ ]:
model.compile(
  optimizer='adam',
  loss='binary_crossentropy', # 이진 분류
  metrics=['accuracy']
)

In [ ]:
# 훈련 데이터
import datetime
log_dir = 'logs\\fit\\' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

x_val = train_data[:10000]
partial_x_train = train_data[10000:]

# 정답 데이터
y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

history = model.fit(
  partial_x_train,
  partial_y_train,
  epochs=40,
  batch_size=512,
  validation_data=(x_val, y_val),
  verbose=1,
  callbacks=tensorboard_callback
)
# loss: 0.0858 - accuracy: 0.9787 (훈련 내용)
# val_loss: 0.3183 - val_accuracy: 0.8808 (검증 내용)

In [ ]:
results = model.evaluate(
  test_data,
  test_labels,
  verbose=2
)

print('Loss =', results[0])
print('Accuracy =', results[1])

In [ ]:
history_dict = history.history

history_dict.keys()

In [ ]:
import matplotlib.pyplot as plt

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']

loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(
  epochs,
  loss,
  'bo',
  label='Training loss'
)

plt.plot(
  epochs,
  val_loss,
  'b',
  label='Validation loss'
)

plt.title('Training and Validation loss')

plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.legend()

plt.show()

In [ ]:
plt.clf() # 기존의 그래프 삭제

plt.plot(
  epochs,
  acc,
  'bo',
  label='Training Accuracy'
)

plt.plot(
  epochs,
  val_acc,
  'b',
  label='Validation Accuracy'
)

plt.title('Training and Validation Accuracy')

plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.legend()

plt.show()